In [1]:
import mcnpy
import sys
import os
from mcnpy.utils import configure_ace_debug_logging

# Configure debug logging
configure_ace_debug_logging(enable=True)  # Set to False to disable debug output

ModuleNotFoundError: No module named 'mcnpy.ace.parse_ace'

In [2]:
# Define all ACE files to test
acefile = 'files/10010_71.02c'
acefile_U235 = 'files/922350_71.02c'
ace1 = 'files/FE56.293600c'
ace2 = 'files/260560_80.02c'

# Print files to verify they exist
for file_path in [acefile, acefile_U235, ace1, ace2]:
    print(f"File {file_path} exists: {os.path.exists(file_path)}")

File files/10010_71.02c exists: True
File files/922350_71.02c exists: True
File files/FE56.293600c exists: False
File files/260560_80.02c exists: False


In [3]:
loc = 5172961
print((loc+1)/4+13-0.25)

1293253.25


In [4]:
ace = mcnpy.read_ace(acefile_U235)

ValueError: Angular distribution locators missing. Call read_angular_locator_blocks first.

In [ ]:
ace

                                   ACE Format Data                                   
                                   Material: 9228                                    
                        ZA: 92235, Temperature: 2.5301e-08 K                         

ACE Data Components:
-------------------------------------------------------------------------------------
Component                                Status                                    
-------------------------------------------------------------------------------------
Header Information                       Loaded (.header)                          
Energy Grid & Cross Sections             Not Loaded                                
Nubar (ν) Data                           Not Loaded                                
Delayed Neutron Data                     Not Loaded                                
Reaction MT Numbers                      Not Loaded                                
Reaction Q-values                        Not

In [ ]:
# ace = mcnpy.read_ace(acefile)
# ace_U235 = mcnpy.read_ace(acefile_U235)

In [ ]:
# ace_U235.angular_distributions

In [ ]:
# ace_U235.angular_distributions.incident_neutron[16]

In [ ]:
# ace_U235.angular_distributions.to_dataframe(energy=2, mt=16)

In [ ]:
# ace_U235.reaction_mt_data

In [ ]:
# ace_U235.energy_distributions

In [ ]:
# Print all neutron reactions with energy distributions
# for mt in ace_U235.energy_distributions.get_neutron_reaction_mt_numbers():
#     distributions = ace_U235.energy_distributions.get_neutron_distribution(mt)
#     for dist in distributions:
#         print(f"MT={mt}, {dist.law_name} (Law {dist.law})")

In [ ]:
# ace.reaction_mt_data

In [ ]:
# ace.particle_release

In [ ]:
# ace_U235.reaction_mt_data.incident_neutron

In [ ]:
# print(f"Elastic locator: {ace.angular_locators.elastic_scattering}")

In [ ]:
# ace.angular_distributions

In [ ]:
# ace_U235.header.nxs_array[3]

In [ ]:
# ace_U235

In [ ]:
# ace_U235.angular_distributions

In [ ]:
# ace_U235.angular_distributions.incident_neutron[16]

In [ ]:
# ace_U235.angular_distributions.incident_neutron[16].plot(16,ace_U235)

In [ ]:
# mu = ace_U235.angular_distributions.elastic.sample_mu(energy=1.0, random_value=0.5)
# mu

In [ ]:
# ace_U235.get_cross_section(1)

In [ ]:
# ace_U235.plot_cross_section(reactions=[1,2,4])

In [ ]:
# write_ace(ace, filepath=acefile+'_recon', overwrite=True)

In [ ]:
# write_ace(ace_U235, acefile_U235+'_recon', overwrite=True)